In [ ]:
pip install transformers

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../interpark.csv', index_col=0)
df

In [ ]:
import ast 
df['rates'] = df['rates'].apply(ast.literal_eval)   
df['rates2'] = df['rates'].apply(lambda x: [5, 5, 5, 5, 5] if sum(x) == 50 else x)

In [ ]:
max_length = 30
batch_size = 64
num_epochs = 3
valid_split = 0.2

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# 데이터셋 가정 (실제 데이터에 맞게 수정 필요)
texts = df['review'].tolist()
labels = df['rates2'].tolist()

# 레이블 수
num_labels = len(labels[0])

# KoBERT 토크나이저 로딩
tokenizer = BertTokenizer.from_pretrained('monologg/kobert', do_lower_case=False)

# 토큰화 함수 정의
def tokenize_texts(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt")

# 데이터를 토큰화
tokenized_texts = tokenize_texts(texts)
labels = torch.tensor(labels, dtype=torch.float)

# TensorDataset 생성
dataset = TensorDataset(tokenized_texts['input_ids'], tokenized_texts['attention_mask'], tokenized_texts['token_type_ids'], labels)

# DataLoader 생성
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 모델 로딩
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=num_labels)

# 옵티마이저 설정
optimizer = AdamW(model.parameters(), lr=5e-5)

# 모델 학습
for epoch in tqdm(range(1)):
    for batch in dataloader:
        input_ids, attention_masks, token_type_ids, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()


In [ ]:
env_name = ['시설(객실/부대시설)',
'위치',
'청결',
'서비스',
'분위기']

def predict_labels(test_texts):
    test_tokenized_texts = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")
    test_outputs = model(**test_tokenized_texts)
    predicted_probs = torch.sigmoid(test_outputs.logits)

    # 결과 출력
    for i, test_text in enumerate(test_texts):
        print(f"입력값: {test_text}")
        for j, prob in enumerate(predicted_probs[i]):
            print(f"{env_name[j]} : {prob.item()*100:.2f}%")
        print("\n")

# 테스트 실행
test_texts = ["아무리 회원이 아니고 숙박앱 통해서 갔다해도 이건 너무하네요.", "경치가 좋은 곳에 가고싶어요."]
predict_labels(test_texts)

입력값: 아무리 회원이 아니고 숙박앱 통해서 갔다해도 이건 너무하네요.
시설(객실/부대시설) : 58.14%
위치 : 68.78%
청결 : 61.07%
서비스 : 49.42%
분위기 : 49.03%


입력값: 경치가 좋은 곳에 가고싶어요.
시설(객실/부대시설) : 57.81%
위치 : 68.32%
청결 : 60.51%
서비스 : 45.64%
분위기 : 49.13%